In [1]:
import pandas as pd
import ast
import utils.helpers as hpr

In [3]:
df = hpr.combine_openstack_data()

In [ ]:
df["reviewers"] = df["reviewers"].apply(ast.literal_eval)

In [ ]:
os_projects = df["project"].unique()

### Casual contributors

In [ ]:
developers = pd.read_csv("./RQs/Files/casual_constributors.csv", index=False)

### How many changes made by core and non. core developers ?

#### Developers

In [65]:
developers.loc[developers["nbr_changes"] == 1, "dev"].to_csv("./RQs/Files/casual_constributors.csv", index=False)

In [66]:
# number of casual developers who performed at most one commit to a swoftware project
len(developers.loc[developers["nbr_changes"] == 1])

2322

In [261]:
os_core_team_devs = pd.read_csv("./RQs/Files/os_core_team.csv")
os_core_team_devs["developers"] = os_core_team_devs["developers"].map(ast.literal_eval)
os_core_team_devs = list(dict.fromkeys(hpr.flatten_list(os_core_team_devs["developers"].values)))

In [ ]:
casual_contributors = developers.loc[developers["nbr_changes"] == 1, "dev"].values

In [61]:
developers.loc[developers["nbr_changes"] == 1, "dev"]

Series([], Name: dev, dtype: int64)

In [31]:
developers = developers.loc[developers["nbr_changes"] != 1]

In [32]:
developers["dev_status"] = developers["dev"].map(lambda x: "Core" if x in os_core_team_devs else "Not core")

In [111]:
developers[["dev_status", "nbr_changes"]].to_csv("./RQs/PQ/Files/core_non_core_changes.csv", index=False)

In [2]:
owners = pd.read_csv("./RQs/PQ/Files/core_non_core_changes.csv")

In [ ]:
owners.loc[(owners["dev_status"] == "Not core")&(owners["nbr_changes"] == 25)]#sum()/owners["nbr_changes"].sum()

In [52]:
len(dict.fromkeys(df.loc[df["owner_account_id"].isin(developers.loc[developers["dev_status"] == "Not core", "dev"].values), "project"]))

1287

#### Reviewers

In [71]:
# Filter non abandoned changes
reviewers = df[df["status"] != "ABANDONED"]
reviewers["reviewers"] = reviewers["reviewers"].apply(ast.literal_eval)

C:\Users\Ali\AppData\Local\Temp/ipykernel_10844/4092083601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewers["reviewers"] = reviewers["reviewers"].apply(ast.literal_eval)


In [72]:
casual_contributors

array([34310, 35458, 34911, ...,   751,  2005,  1657], dtype=int64)

In [73]:
reviewers["reviewers_ids"] = reviewers["reviewers"].map(lambda revs: [
    rev["_account_id"] for rev in revs
    if ("tags" not in rev) and (rev["_account_id"] not in casual_contributors)
])

C:\Users\Ali\AppData\Local\Temp/ipykernel_10844/792228773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewers["reviewers_ids"] = reviewers["reviewers"].map(lambda revs: [


In [74]:
reviewers = reviewers[["number", "status", "project", "reviewers_ids"]]

In [75]:
reviewers["revs_length"] = reviewers["reviewers_ids"].map(lambda x: len(x))

In [76]:
reviewers = reviewers[reviewers["revs_length"] != 0]
reviewers = reviewers.explode(column="reviewers_ids")
reviewers.rename(columns={"reviewers_ids": "rev_id"}, inplace=True)
reviewers["count"] = 0
reviewers = reviewers.groupby(by="rev_id").count().reset_index(level=0)[["rev_id", "count"]]
reviewers["rev_status"] = reviewers["rev_id"].map(lambda rev_id: "Core" if rev_id in os_core_team_devs else "Not core")
reviewers.reset_index(drop=True, inplace=True)

In [97]:
reviewers[(reviewers["rev_status"] == "Core")&(reviewers["rev_id"]==3153)]#.sort_values(by="count", ascending=0)

,rev_id,count,rev_status
490,3153,32961,Core


In [48]:
reviewers[["rev_status", "count"]].to_csv("./RQs/PQ/Files/core_non_core_reviewers_changes.csv", index=False)

### Analysis of OS teams release notes' projects overlap

In [58]:
request_teams = rq.get("https://releases.openstack.org/index.html#teams")
soup_teams = BeautifulSoup(request_teams.text, 'html.parser')

teams_projects = {}
section_teams = soup_teams.find("section", attrs={"id": "teams"})
li_teams = section_teams.find_all("li", "toctree-l1")

for i in range(0, len(li_teams)):
    li_segment = li_teams[i].find("a")
    link = li_teams[i].find("a")
    link_suffix = link["href"]

    request_projects = rq.get("https://releases.openstack.org/%s" % (link_suffix))
    soup_projects = BeautifulSoup(request_projects.text, 'html.parser')
    table_of_contents = soup_projects.find("div", attrs={"id": "table-of-contents"})
    ul_projects = table_of_contents.find("ul")
    ul_projects = ul_projects.find("ul")
    ul_projects = ul_projects.find_all("ul")
    projects = []
    for ul_item in ul_projects:

        href_projects = ul_item.find_all("a")

        for href_project in href_projects:
            projects.append(href_project.text.replace(" (EOL)", ""))
    #         href_links_projects = soup_projects.find_all("a", "reference internal")
    teams_projects[link_suffix[6:-5]] = list(dict.fromkeys(projects))
    # projects = [href_links_projects[j].text for j in range(3, len(href_links_projects))]
    # teams_deliverables[link_suffix[:-5].replace("-", "")] = projects

In [59]:
df_teams_projects = pd.DataFrame({"team": teams_projects.keys(), "projects": teams_projects.values()})

In [60]:
def get_intersection_percentage(df, team_a, team_b):
    projects_a = df.loc[df["team"] == team_a, "projects"].values.tolist()[0]
    projects_b = df.loc[df["team"] == team_b, "projects"].values.tolist()[0]
    intersect_team = list(set(projects_a).intersection(projects_b))
    union_team = list(set(projects_a).union(projects_b))
    return (len(intersect_team) / len(union_team)) * 100

def get_teams_metrics(df, teams):
    result = []
    treated_items = []
    for i in range(len(teams)):
        team_a = teams[i]
        for j in range(len(teams)):
            team_b = teams[j]
            if (team_a != team_b) and (team_b not in treated_items):
                
                new_item = {
                    "teamA": team_a,
                    "teamB": team_b,
                    "intersection_percentage": get_intersection_percentage(df, team_a, team_b)
                }
                result.append(new_item)
            
        treated_items.append(team_a)

    return result

In [61]:
teams = df_teams_projects["team"].values.tolist()
teams_metrics = get_teams_metrics(df_teams_projects, teams)
df_teams_metrics = pd.DataFrame(teams_metrics)

In [62]:
def get_nbr_common_projects(row):
    projects_a = df_teams_projects.loc[df_teams_projects["team"] == row["teamA"], "projects"].values.tolist()[0]
    projects_b = df_teams_projects.loc[df_teams_projects["team"] == row["teamB"], "projects"].values.tolist()[0]

    return len(set(projects_a).intersection(projects_b))

def get_common_projects(row):
    projects_a = df_teams_projects.loc[df_teams_projects["team"] == row["teamA"], "projects"].values.tolist()[0]
    projects_b = df_teams_projects.loc[df_teams_projects["team"] == row["teamB"], "projects"].values.tolist()[0]

    return list(set(projects_a).intersection(projects_b))

In [63]:
df_teams_metrics["projects_count"] = df_teams_metrics[["teamA", "teamB"]].apply(get_nbr_common_projects, axis=1)
df_teams_metrics["projects_common"] = df_teams_metrics[["teamA", "teamB"]].apply(get_common_projects, axis=1)
# df_teams_metrics.to_csv("./RQs/PQ/Files/teams_metrics.csv", index=False)

In [64]:
df_teams_metrics.sort_values(by="projects_count", ascending=0, inplace=True)
df_teams_metrics.to_csv("./RQs/PQ/Files/teams_metrics.csv", index=False)

In [66]:
df_teams_metrics = pd.read_csv("./RQs/PQ/Files/teams_metrics.csv")

In [73]:
len(df_teams_metrics[df_teams_metrics["intersection_percentage"] == 0]), len(df_teams_metrics)

(1272, 1275)

In [14]:
# df_teams_metrics.groupby(by="projects_count").count().reset_index(level=0)[["projects_count", "intersection_percentage"]].to_csv("./RQs/PQ/Files/teams_projects_count.csv", index=False)#sort_values(by="intersection_percentage", ascending=0)
df_teams_metrics.groupby(by="intersection_percentage").count().reset_index(level=0)[["projects_count", "intersection_percentage"]].to_csv("./RQs/PQ/Files/teams_projects_percentage.csv", index=False)#sort_values(by="intersection_percentage", ascending=0)

### Analysis for OS projects pairs

In [67]:
def combine_projects_pairs_metrics():
    '''Combine generated csv files into a single DataFrame object
    '''
    df = pd.DataFrame([])
    data_path = "%sRQs/PQ/Files/projects_metrics/" % hpr.DIR
    changes_file_names = hpr.list_file(data_path)
    for f in changes_file_names:
        df_per_file = pd.read_csv("%s%s" % (data_path, f))
        df = pd.concat((df, df_per_file))

    return df

In [87]:
df_projects_pairs = combine_projects_pairs_metrics()

In [217]:
df_projects_metrics = df_projects_pairs[df_projects_pairs["projectA"].str.startswith("openstack/") & df_projects_pairs["projectB"].str.startswith("openstack/")]

#### all intersection

In [219]:
df_projects_metrics = df_projects_metrics.groupby(by=["projectA", "projectB"]).sum().reset_index(level=[0,1])#[["intersect_dev", "intersect_rev"]].to_csv("./RQs/PQ/Files/projects_metrics.csv", index=False)

In [231]:
# len(
# df_projects_metrics[df_projects_metrics["revAB_count"]>0].median()
df_projects_pairs[(df_projects_pairs["developer"]=="Non core")&(df_projects_pairs["intersect_rev"]>0)].median()
    # )/len(df_projects_metrics)#.median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_1469/2755803569.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_projects_pairs[(df_projects_pairs["developer"]=="Non core")&(df_projects_pairs["intersect_rev"]>0)].median()


intersect_dev    0.124069
intersect_rev    2.222222
devAB_count      2.000000
revAB_count      2.000000
dtype: float64

In [118]:
df_projects_pair_labels = df_projects_metrics.groupby(by=["pair"]).sum().reset_index(level=[0])
# df_projects_pair_labels = df_projects_pair_labels.loc[df_projects_metrics["intersect_rev"] > 0, "pair"]
df_projects_pair_labels

,pair,intersect_dev,intersect_rev,devAB_count,revAB_count
0,openstack/activity-board openstack/activity-board,83.636841,639.157645,50,684
1,openstack/adjutant openstack/adjutant,5103.680455,5560.576934,4974,6894
2,openstack/adjutant-ui openstack/adjutant-ui,3053.979734,4744.503546,1844,3844
3,openstack/anchor openstack/anchor,53.541343,753.531980,48,742
4,openstack/ansible-collection-kolla openstack/a...,286.019433,633.366376,288,1044
...,...,...,...,...,...
1284,openstack/zaqar-ui openstack/zaqar-ui,6545.926096,5486.051081,8736,10802
1285,openstack/zun openstack/zun,1950.881696,2032.129931,6578,9782
1286,openstack/zun-tempest-plugin openstack/zun-tem...,3725.999793,4297.643599,3640,5520
1287,openstack/zun-ui openstack/zun-ui,4509.265039,3470.311347,5854,8092


In [113]:
# df_projects_pair_labels["pair"] = df_projects_pair_labels.
df_projects_metrics
# df_projects_pairs[(df_projects_pairs["developer"]=="Non core")&(df_projects_pairs["intersect_rev"]>0)].median()

,projectA,projectB,developer,intersect_dev,intersect_rev,devAB_count,revAB_count,pair
0,openstack/deb-python-dogpile.cache,openstack/deb-python-futurist,Core,33.333333,0.000000,3,4,openstack/deb-python-dogpile.cache openstack/d...
1,openstack/deb-python-dogpile.cache,openstack/deb-python-futurist,Non core,66.666667,100.000000,3,4,openstack/deb-python-dogpile.cache openstack/d...
2,openstack/deb-python-dogpile.cache,openstack/deb-python-glanceclient,Core,33.333333,0.000000,3,4,openstack/deb-python-dogpile.cache openstack/d...
3,openstack/deb-python-dogpile.cache,openstack/deb-python-glanceclient,Non core,66.666667,80.000000,3,4,openstack/deb-python-dogpile.cache openstack/d...
4,openstack/deb-python-dogpile.cache,openstack/deb-python-glance-store,Core,33.333333,0.000000,3,4,openstack/deb-python-dogpile.cache openstack/d...
...,...,...,...,...,...,...,...,...
1185,openstack/mistral-specs,openstack/magnetodb-specs,Non core,0.000000,3.508772,1,2,openstack/mistral-specs openstack/mistral-specs
1196,openstack/mistral-specs,openstack/magnetodb,Core,1.960784,0.000000,1,2,openstack/mistral-specs openstack/mistral-specs
1197,openstack/mistral-specs,openstack/magnetodb,Non core,0.000000,2.631579,1,2,openstack/mistral-specs openstack/mistral-specs
1198,openstack/mistral-specs,openstack/openstack-nose,Core,0.000000,0.000000,0,1,openstack/mistral-specs openstack/mistral-specs


In [92]:
project_pairs = ["%s %s" % (p[0], p[1]) for p in project_pairs]

In [96]:
df_projects_metrics["pair"] = df_projects_metrics[["projectA", "projectB"]].apply(lambda pair: "%s %s" % (pair["projectA"], pair["projectA"]), axis=1)

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_1469/3025848089.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_projects_metrics["pair"] = df_projects_metrics[["projectA", "projectB"]].apply(lambda pair: "%s %s" % (pair["projectA"], pair["projectA"]), axis=1)


In [102]:
df_projects_metrics[(df_projects_metrics["developer"]=="Non core")&(df_projects_metrics["pair"].isin(project_pairs))]#.median()

,projectA,projectB,developer,intersect_dev,intersect_rev,devAB_count,revAB_count,pair


In [20]:
df_projects_metrics = pd.read_csv("./RQs/PQ/Files/projects_metrics.csv")

In [74]:
# df_projects_pairs[(df_projects_pairs["developer"]=="Non core")&(df_projects_pairs["intersect_dev"]>0)].median()
# len(
# df_projects_metrics.loc[df_projects_metrics["intersect_rev"]>0]#.median()
# )/len(df_projects_metrics)
df_projects_pairs.loc[(df_projects_pairs["developer"]=="Non core")&(df_projects_pairs["intersect_rev"]>0)].median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_20657/511374646.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_projects_pairs.loc[(df_projects_pairs["developer"]=="Non core")&(df_projects_pairs["intersect_rev"]>0)].median()


intersect_dev    0.124069
intersect_rev    2.222222
devAB_count      2.000000
revAB_count      2.000000
dtype: float64

### Developer intersection

In [15]:
df_developers_projects_pairs = df_projects_metrics[["projectA", "projectB", "developer", "intersect_dev"]].copy()

In [16]:
df_developers_projects_pairs[["developer", "intersect_dev"]].copy().to_csv("./RQs/PQ/Files/developers_projects_intersection.csv", index=False)

In [18]:
df_projects_metrics = pd.read_csv("./RQs/PQ/Files/developers_projects_intersection.csv")

In [31]:
# len(df_projects_metrics.loc[df_projects_metrics["intersect_dev"]>0])/len(df_projects_metrics)
df_projects_metrics.median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_20657/2822367421.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_projects_metrics.median()


intersect_dev    1.515152
intersect_rev    2.108963
devAB_count      2.000000
revAB_count      2.000000
dtype: float64

### Reviewers intersection

In [17]:
df_reviewers_projects_pairs = df_projects_metrics[["developer", "intersect_rev"]].copy()
df_reviewers_projects_pairs[["intersect_rev", "developer"]].to_csv("./RQs/PQ/Files/reviewers_project_intersection.csv", index=False)

In [59]:
dev = pd.read_csv("./RQs/PQ/Files/reviewers_project_intersection.csv")

In [61]:
dev.loc[dev["developer"]=="Core"].median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_9560/1052162431.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dev.loc[dev["developer"]=="Core"].median()


intersect_rev    0.0
dtype: float64

### Analysis for services

In [252]:
df_services_metrics = pd.read_csv("./RQs/PQ/Files/pq_services_metrics.csv")

In [233]:
df_services_metrics.groupby(by=["serviceA", "serviceB"]).sum().reset_index(level=[0,0])[["intersect_dev", "intersect_rev"]].to_csv("./RQs/PQ/Files/services_metrics.csv", index=False)

In [245]:
df_services_metrics = df_services_metrics.groupby(by=["serviceA", "serviceB"]).sum().reset_index(level=[0,1])
# df_services_metrics["devAB_intersect"] = df_services_metrics["devAB_intersect"].apply(ast.literal_eval)

In [251]:
df_services_metrics[df_services_metrics["intersect_rev"]>0].median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_1469/1934662545.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_services_metrics[df_services_metrics["intersect_rev"]>0].median()


intersect_dev    10.117155
intersect_rev    10.846687
dtype: float64

In [46]:
df_services_metrics["revAB_intersect_count"] = df_services_metrics["revAB_intersect"].apply(ast.literal_eval)
# df_services_metrics = df_services_metrics[df_services_metrics["developer"]=="Core"]

In [13]:
df_services_metrics["devAB_count"] = df_services_metrics["devAB_intersect"].map(lambda x: len(x))

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_1469/1054539711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_services_metrics["devAB_count"] = df_services_metrics["devAB_intersect"].map(lambda x: len(x))


In [47]:
df_services_metrics["revAB_intersect_count"] = df_services_metrics["revAB_intersect"].map(lambda x: len(x))

In [259]:
df_services_metrics[(df_services_metrics["developer"]=="Non core")&(df_services_metrics["intersect_rev"]>0)].median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_1469/293948316.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_services_metrics[(df_services_metrics["developer"]=="Non core")&(df_services_metrics["intersect_rev"]>0)].median()


intersect_dev    5.456801
intersect_rev    6.548570
dtype: float64

In [22]:
# df_services_metrics[df_services_metrics["devAB_count"]>0].median()
df_services_metrics

,serviceA,serviceB,devA,devB,devAB_intersect,devAB_union,revA,revB,revAB_intersect,revAB_union,developer,intersect_dev,intersect_rev
0,nova,zun,"[7634, 4690, 20733, 935, 10366, 11090, 34860, ...","[34329, 8556, 11536, 17685, 29313, 30491, 2066...","[1, 2, 15888, 27153, 24, 29721, 11805, 27678, ...","[1, 2, 3, 4, 5, 6, 7, 12297, 12299, 8205, 8213...","[7166, 8556, 15334, 19234, 20733, 32029, 32238...","[11536, 8556, 16352, 17685, 20663, 29313, 1190...","[1, 2, 3, 19457, 27654, 5638, 21513, 21515, 26...","[1, 2, 3, 4, 16389, 6, 7, 5, 8205, 8213, 24598...",Core,2.605092,2.626050
1,nova,zun,"[7634, 4690, 20733, 935, 10366, 11090, 34860, ...","[34329, 8556, 11536, 17685, 29313, 30491, 2066...","[1, 2, 15888, 27153, 24, 29721, 11805, 27678, ...","[1, 2, 3, 4, 5, 6, 7, 12297, 12299, 8205, 8213...","[7166, 8556, 15334, 19234, 20733, 32029, 32238...","[11536, 8556, 16352, 17685, 20663, 29313, 1190...","[1, 2, 3, 19457, 27654, 5638, 21513, 21515, 26...","[1, 2, 3, 4, 16389, 6, 7, 5, 8205, 8213, 24598...",Non core,3.552398,3.711485
2,nova,ironic,"[7634, 4690, 20733, 935, 10366, 11090, 34860, ...","[10342, 32553, 15519, 23851, 28806, 27909, 102...","[1, 2, 3, 6159, 14352, 8213, 6167, 24, 4146, 1...","[1, 2, 3, 4, 5, 6, 7, 20481, 12299, 8205, 8213...","[7166, 8556, 15334, 19234, 20733, 32029, 32238...","[6618, 10342, 11655, 23851, 10239, 27909, 2880...","[1, 2, 3, 4, 7, 6159, 8213, 6167, 24, 10263, 1...","[1, 2, 3, 4, 16389, 6, 7, 5, 8205, 8213, 24598...",Core,7.258065,6.968750
3,nova,ironic,"[7634, 4690, 20733, 935, 10366, 11090, 34860, ...","[10342, 32553, 15519, 23851, 28806, 27909, 102...","[1, 2, 3, 6159, 14352, 8213, 6167, 24, 4146, 1...","[1, 2, 3, 4, 5, 6, 7, 20481, 12299, 8205, 8213...","[7166, 8556, 15334, 19234, 20733, 32029, 32238...","[6618, 10342, 11655, 23851, 10239, 27909, 2880...","[1, 2, 3, 4, 7, 6159, 8213, 6167, 24, 10263, 1...","[1, 2, 3, 4, 16389, 6, 7, 5, 8205, 8213, 24598...",Non core,12.903226,16.406250
4,nova,cyborg,"[7634, 4690, 20733, 935, 10366, 11090, 34860, ...","[8556, 26458, 35058, 17685, 31412, 31827, 2395...","[11904, 1, 2, 27781, 25738, 30092, 23950, 2331...","[1, 2, 3, 4, 5, 6, 7, 12299, 8205, 8213, 24598...","[7166, 8556, 15334, 19234, 20733, 32029, 32238...","[26458, 31412, 23950, 8556, 17685, 30396, 3202...","[2, 3, 7, 18955, 11278, 26126, 27153, 28182, 1...","[1, 2, 3, 4, 16389, 6, 7, 5, 8205, 8213, 24598...",Core,1.631420,2.163888
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655,telemetry,venus,"[32240, 35067, 9816, 13177, 34120, 17685, 3505...","[34375, 22629, 31412, 30562, 35058, 33436, 264...","[28522, 8556, 27565, 30092, 30384, 35058, 2645...","[1, 2, 19457, 3, 21515, 14347, 8205, 6159, 821...","[4264, 30396, 7294, 28543, 32029, 32238, 8556,...","[26458, 31412, 27565, 34375, 24061, 32238, 176...","[13252, 14151, 28743, 8556, 27565, 32238, 2628...","[1, 2, 3, 4, 19457, 9218, 11268, 21513, 14347,...",Non core,1.246106,1.242236
3656,telemetry,winstackers,"[32240, 35067, 9816, 13177, 34120, 17685, 3505...","[8543, 7634, 35067, 35058, 17685, 33881, 33436...","[11904, 1, 16896, 2, 27781, 28935, 21387, 6159...","[1, 2, 19457, 3, 10241, 21515, 14347, 8205, 61...","[4264, 30396, 7294, 28543, 32029, 32238, 8556,...","[8543, 8556, 17685, 7634, 11805, 8213, 3185, 2...","[11904, 20993, 3, 27781, 28935, 19339, 19853, ...","[1, 2, 3, 4, 19457, 9218, 11268, 21513, 14347,...",Core,6.162465,3.082852
3657,telemetry,winstackers,"[32240, 35067, 9816, 13177, 34120, 17685, 3505...","[8543, 7634, 35067, 35058, 17685, 33881, 33436...","[11904, 1, 16896, 2, 27781, 28935, 21387, 6159...","[1, 2, 19457, 3, 10241, 21515, 14347, 8205, 61...","[4264, 30396, 7294, 28543, 32029, 32238, 8556,...","[8543, 8556, 17685, 7634, 11805, 8213, 3185, 2...","[11904, 20993, 3, 27781, 28935, 19339, 19853, ...","[1, 2, 3, 4, 19457, 9218, 11268, 21513, 14347,...",Non core,6.722689,6.358382
3658,venus,winstackers,"[34375, 22629, 31412, 30562, 35058, 33436, 264...","[8543, 7634, 35067, 35058, 17685, 33881, 33436...","[32326, 28522,

In [32]:
df_services_metrics = pd.read_csv("./RQs/PQ/Files/services_metrics.csv")

In [ ]:
# len(
df_services_metrics[(df_services_metrics["intersect_dev"]>0)&(df_services_metrics["developer"]=="Non core")].median()
    # )/len(df_services_metrics)

In [64]:
df_services_metrics.sort_values(by="intersect_rev", ascending=1)

,intersect_dev,intersect_rev
1443,0.000000,0.341297
1436,0.000000,0.393701
1076,0.306373,0.569801
1086,0.796081,0.608665
1442,0.299103,0.615764
...,...,...
1607,30.444126,38.058341
1332,33.890215,38.151042
1303,27.919567,38.220919
1337,30.281690,38.672351


In [79]:
# len(df_services_metrics.loc[df_services_metrics.intersect_rev > 0])/len(df_services_metrics)
df_services_metrics.loc[(df_services_metrics["developer"]=="Core")&(df_services_metrics["intersect_rev"]>0)].median()
# df_services_metrics.loc[df_services_metrics.intersect_rev > 0].median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_20657/1775384484.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_services_metrics.loc[(df_services_metrics["developer"]=="Core")&(df_services_metrics["intersect_rev"]>0)].median()


intersect_dev    4.421250
intersect_rev    4.356785
dtype: float64

#### Developer

In [43]:
df_owners_services_pairs = df_services_metrics[["serviceA", "serviceB", "developer", "intersect_dev"]].copy()

In [131]:
df_owners_services_pairs[["developer", "intersect_dev"]].copy().to_csv("./RQs/PQ/Files/owners_service_intersection.csv", index=False)

In [58]:
df_owners_services_pairs = pd.read_csv("./RQs/PQ/Files/owners_service_intersection.csv")

#### Reviewers

In [22]:
df_reviewers_services_pairs = df_services_metrics[["developer", "intersect_rev"]].copy()

In [136]:
df_reviewers_services_pairs[["intersect_rev", "developer"]].to_csv("./RQs/PQ/Files/reviewers_service_intersection.csv", index=False)

In [56]:
df_reviewers_services_pairs = pd.read_csv("./RQs/PQ/Files/reviewers_service_intersection.csv")

### # of developers per # of projects

In [6]:
casual_contributors = pd.read_csv("./RQs/Files/casual_contributors.csv")["dev"].values

In [263]:
df_dev_per_projects = df.loc[(df["is_owner_bot"] == 0)&(df["status"] == "MERGED")&(~df["owner_account_id"].isin(casual_contributors))].groupby(by=["owner_account_id"]).agg({"project": pd.Series.nunique}).reset_index(level=0)#.groupby("project").count().reset_index(level=0)
df_dev_per_projects.rename(columns={"project": "projects_nbr", "owner_account_id": "devs_nbr"}, inplace=True)
df_dev_per_projects["dev_status"] = df_dev_per_projects["devs_nbr"].map(lambda rev_id: "Core" if rev_id in os_core_team_devs else "Non core")
df_dev_per_projects.to_csv("./RQs/PQ/Files/devs_per_nbr_project.csv", index=False)

In [272]:
# len(df_dev_per_projects[df_dev_per_projects["projects_nbr"]==1])/len(df_dev_per_projects)
df_dev_per_projects[df_dev_per_projects["dev_status"]=="Non core"].median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_1469/1785655894.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dev_per_projects[df_dev_per_projects["dev_status"]=="Non core"].median()


devs_nbr        17104.0
projects_nbr        3.0
dtype: float64

### # of reviewers per # of projects

In [273]:
df_rev_per_projects = df.loc[(df["status"] == "MERGED")&(~df["owner_account_id"].isin(casual_contributors))].copy()

In [52]:
df_rev_per_projects["reviewers"] = df_rev_per_projects["reviewers"].map(lambda revs: [rev["_account_id"] for rev in revs if ("tags" not in rev) and (rev["_account_id"] not in casual_contributors)])

In [53]:
df_rev_per_projects = df_rev_per_projects[["project", "number", "reviewers"]].explode(column="reviewers")

In [54]:
df_rev_per_projects = df_rev_per_projects.groupby(by=["reviewers"]).agg({"project": pd.Series.nunique}).reset_index(level=0)#.groupby("project").count().reset_index(level=0)
df_rev_per_projects.rename(columns={"project": "projects_nbr", "reviewers": "reviewer"}, inplace=True)
df_rev_per_projects["rev_status"] = df_rev_per_projects["reviewer"].map(lambda rev_id: "Core" if rev_id in os_core_team_devs else "Non core")
df_rev_per_projects.to_csv("./RQs/PQ/Files/revs_per_nbr_project.csv", index=False)

In [274]:
df_rev_per_projects = pd.read_csv("./RQs/PQ/Files/revs_per_nbr_project.csv")

In [282]:
# len(
df_rev_per_projects[df_rev_per_projects["rev_status"]=="Non core"]#.median()
# )/len(df_rev_per_projects)

,reviewer,projects_nbr,rev_status
2,3,1019,Non core
4,5,9,Non core
5,6,20,Non core
6,7,95,Non core
7,8,3,Non core
...,...,...,...
8991,35191,2,Non core
8992,35199,3,Non core
8993,35240,1,Non core
8994,35266,1,Non core


### # of developers per # of services

In [59]:
online_repositories = pd.read_csv("./all_os_components.csv")

online_repositories["related_projects"] = online_repositories["related_projects"].apply(ast.literal_eval)

online_repositories = dict(zip(online_repositories["main_project"].values.reshape(-1), online_repositories["related_projects"].values))

def invert_projects_services():
    return {"openstack/%s" % p:service for service, projects in online_repositories.items() for p in projects}

project_services_inv = invert_projects_services()

In [62]:
df_dev_per_services = df.loc[(df["is_owner_bot"] == 0)&(df["status"] == "MERGED")&(~df["owner_account_id"].isin(casual_contributors)), ["project", "owner_account_id"]]
df_dev_per_services["service"] = df_dev_per_services["project"].map(lambda p: project_services_inv.get(p))

In [63]:
len(df_dev_per_services[df_dev_per_services["service"].isnull()])

90780

In [64]:
df_dev_per_services.dropna(subset=["service"], inplace=True)
df_dev_per_services = df_dev_per_services.groupby(by=["owner_account_id"]).agg({"service": pd.Series.nunique}).reset_index(level=0)#.groupby("project").count().reset_index(level=0)
df_dev_per_services["dev_status"] = df_dev_per_services["owner_account_id"].map(lambda rev_id: "Core" if rev_id in os_core_team_devs else "Non core")
df_dev_per_services.to_csv("./RQs/PQ/Files/devs_per_nbr_service.csv", index=False)

In [2]:
df_dev_per_projects = pd.read_csv("./RQs/PQ/Files/devs_per_nbr_project.csv")
df_dev_per_services = pd.read_csv("./RQs/PQ/Files/devs_per_nbr_service.csv")

In [10]:
len(df_dev_per_services[df_dev_per_services["dev_status"]=="Core"])/len(df_dev_per_services)
# df_dev_per_services

0.11173184357541899

In [284]:
developers = pd.read_csv("./RQs/PQ/Files/devs_per_nbr_service.csv")
reviewers = pd.read_csv("./RQs/PQ/Files/revs_per_nbr_service.csv")

In [302]:
# len(reviewers[reviewers["service"]==1])/len(reviewers)
reviewers[(reviewers["rev_status"]=="Non core")].median()
# len(developers[(developers["service"]==1)&(developers["dev_status"]=="Non core")])/len(developers[developers["dev_status"]=="Non core"])#.median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_1469/864720424.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  reviewers[(reviewers["rev_status"]=="Non core")].median()


reviewer    17169.0
service         2.0
dtype: float64

### # of reviewers per # of services

In [65]:
df_rev_per_services = df.loc[(df["status"] == "MERGED")&(~df["owner_account_id"].isin(casual_contributors))].copy()
df_rev_per_services["service"] = df_rev_per_services["project"].map(lambda p: project_services_inv.get(p))
df_rev_per_services = df_rev_per_services.explode(column=["service"])
df_rev_per_services.dropna(subset=["service"], inplace=True)
df_rev_per_services["reviewer"] = df_rev_per_services["reviewers"].map(lambda revs: [rev["_account_id"] for rev in revs if ("tags" not in rev) and (rev["_account_id"] not in casual_contributors)])
df_rev_per_services = df_rev_per_services.explode(column=["reviewer"])
df_rev_per_services = df_rev_per_services.groupby(by=["reviewer"]).agg({"service": pd.Series.nunique}).reset_index(level=0)#.groupby("project").count().reset_index(level=0)
df_rev_per_services["rev_status"] = df_rev_per_services["reviewer"].map(lambda rev_id: "Core" if rev_id in os_core_team_devs else "Non core")
df_rev_per_services.to_csv("./RQs/PQ/Files/revs_per_nbr_service.csv", index=False)

### median of ranges of services in terms of developers' intersection that partipate in each project pair within the same service

In [7]:
def retrieve_intersect(list1, list2):
    return list(set(list1).intersection(list2))

def retrieve_union(list1, list2):
    return list(set(list1).union(list2))

def retrieve_services_medians():
    openstack_services = pd.read_csv("./all_os_components.csv")
    openstack_services["related_projects"] = openstack_services["related_projects"].map(ast.literal_eval)

    result = dict.fromkeys(openstack_services["main_project"].values)

    for idx, row in openstack_services.iterrows():
        service = row["main_project"]
        related_projects = row["related_projects"]
        treated_items = []
        project_intersections = []
        for i in range(len(related_projects)):
            projectA = related_projects[i]
            for j in range(len(related_projects)):
                projectB = related_projects[j]
                if (projectA != projectB) and (projectB not in treated_items):
                    devsA = df.loc[(df["project"] == "openstack/%s" % (projectA))&(df["is_owner_bot"]==False)&(~df["owner_account_id"].isin(casual_contributors)), "owner_account_id"].values
                    devsB = df.loc[(df["project"] == "openstack/%s" % (projectB))&(df["is_owner_bot"]==False)&(~df["owner_account_id"].isin(casual_contributors)), "owner_account_id"].values

                    common_devs = len(retrieve_union(devsA, devsB)) if len(retrieve_union(devsA, devsB)) != 0 else 1
                    devAB_intersect = len(retrieve_intersect(devsA, devsB)) * 100 / common_devs
                    project_intersections.append(devAB_intersect)
            treated_items.append(projectA)
        # result[service] = statistics.median(project_intersections) if len(project_intersections) != 0 else 0
        result[service] = project_intersections
    # return pd.DataFrame({"service": result.keys(), "median": result.values()})
    return result
     

In [8]:
df_services_medians = retrieve_services_medians()

In [33]:
df_services_medians = pd.DataFrame({"service": df_services_medians.keys(), "median": df_services_medians.values()})
df_services_medians = df_services_medians.explode(column="median")
df_services_medians.dropna(subset="median", inplace=True)
df_services_medians.reset_index(drop=True, inplace=True)
df_services_medians.to_csv("./RQs/PQ/Files/all_services_medians.csv", index=False)

In [40]:
chain = [864384, 861571, 861572, 861319, 861320, 862219, 862479, 861208, 861467, 862492, 861471, 861609, 863532, 864946, 864948, 861110, 861111, 861366, 861369, 861116, 861635, 862797, 862798, 862799, 861136, 861137, 862801, 862803, 861140, 862805, 862806, 862802, 862809, 862810, 862811, 862812, 862814, 862816, 862820, 861158, 862183, 862189, 860795, 861053]

In [42]:
len(df.loc[df["number"].isin(chain), "project"].unique())

40